In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
# Reading xlsx file
df = pd.ExcelFile('data (1).xlsx')

output = df.parse('OUTPUT')
fields = df.parse('Field key')

In [4]:
# Remove the null rows
fields = fields.dropna()

In [5]:
# giving 3rd column a name
fields.columns = ['Data Field', 'Hazard', 'Type']

In [6]:
# test if we can get the matchingfield 
matching_dataField = fields.loc[fields['Type'].str.contains('Flam liquid')].reset_index().iloc[0, 1]

In [7]:
matching_dataField

'southafrica_ghs-2.6_display'

In [ ]:
fields

## Dealing with output now

In [25]:
output = df.parse('OUTPUT')


In [26]:
# dropping first row(example)
output = output.drop(0).reset_index(drop=True)

In [ ]:
output

In [29]:
output['Hazard class'] = output['Hazard class'].str.replace('\n', ' ')

In [ ]:
mapping = []
for i in range(output.shape[0]):
    mapping.append([])
    #checking for non null values
    if output.loc[i, 'Hazard class'] != 'NaN':
        #if not null then replace the value with the matching data field
        try:
            for hazard in output.loc[i, 'Hazard class'].split(','): 
                m = re.search(r'\d', hazard)
                if m:
                    hazard_name = hazard[:m.start()].strip()
                    matching_dataField = fields.loc[fields['Type'].str.contains(
                        hazard_name, case = False, regex = True)].reset_index().iloc[0, 1]
                    hazard_category = hazard[m.start():].strip()
                    output.loc[i, matching_dataField] = "Category " + hazard_category
                    
                    mapping[i].append(hazard_name + " : " + matching_dataField)
                else:
                    hazard_name = hazard.strip()
                    matching_dataField = fields.loc[fields['Type'].str.contains(hazard_name)].reset_index().iloc[0, 1]
                    output.loc[i, matching_dataField] = "Yes"

                    mapping[i].append(hazard_name + " : " + matching_dataField)
        except:
            pass
            
                


In [50]:
output.to_excel('output.xlsx')
    


In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in mapping]))


In [56]:
with open('checking.txt', 'w') as file:
    i = 1
    for row in mapping:
        # Store the row number in the file
        file.write(str(i) + ' : ')
        file.write(' '.join([str(item) for item in row]))
        file.write('\n')
        i += 1
